In [ ]:
# !pip install transformers datasets accelerate torch pandas

In [ ]:

import torch
import os
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from src.data_tokenizer import RapDataTokenizer, load_and_process_csv

In [ ]:
DATA_URL = 'https://raw.githubusercontent.com/ivanchetvergov/neiroRap/main/data/lyrics_df.csv'
LOCAL_CSV = 'data/lyrics_df.csv'
MODEL_NAME = "sberbank-ai/rugpt3small_based_on_gpt2"

# параметры обучения
BLOCK_SIZE = 1024
BATCH_SIZE = 4
EPOCHS = 11
LEARNING_RATE = 2e-6

# выходные директории
OUTPUT_DIR = "./neiroRap_results_v2"
FINAL_MODEL_DIR = "./neiroRap_final_model_v2"
LOGS_DIR = './neiroRap_logs_v2'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [ ]:
rap_tokenizer = RapDataTokenizer(MODEL_NAME)

print(f"\nИнформация о токенизаторе:")
print(f"   • Базовая модель: {MODEL_NAME}")
print(f"   • Размер словаря: {len(rap_tokenizer.tokenizer)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Добавлено 23 специальных токенов
Размер словаря: 50280

Информация о токенизаторе:
   • Базовая модель: sberbank-ai/rugpt3small_based_on_gpt2
   • Размер словаря: 50280


In [ ]:
df_processed = load_and_process_csv(LOCAL_CSV, rap_tokenizer)
print(f"=== Пример текста: === \n{df_processed['tokenized_text'].iloc[0][:500]}")

Загрузка файла: data/lyrics_df.csv
Загружено 2042 строк
Колонки: index, artist, title, lyrics, structured_text, tags, primary_artist, album, tempo, energy, valence, danceability
Обработка датафрейма...
Обработано строк: 2042
Средняя длина текста: 1506 символов
=== Пример текста: === 
<META> [Кащенко (Kaschenko)] [Boulevard Depo] <TM_MED> <EN_MED> <VAL_MED> <DNC_MED> </META>
<LYRICS>
<SEC><INTRO>
PowerpuffLuv, я, эй Boulevard Depo Свэг, пр р У, у
</SEC>
<SEC><VERSE>
PowerpuffLuv и каждый под high kick Папа был прав Luv стал такой тактик Наблюдай, как я стал такой практик Смерть держит в когтях мой огромный, my dick Две сотни лет уже я выгляжу как мальчик Душу не продал, не был этим озадачен Нет, я не Ратмир, и всё в этот раз иначе Выпил таблетки, ищите меня в Кащенко
</SEC>
<


In [ ]:
dataset_dict = {'text': df_processed['tokenized_text'].tolist()}
dataset = Dataset.from_dict(dataset_dict)

In [ ]:
def tokenize_function(examples):
    """Токенизация текстов с truncation"""
    return rap_tokenizer.tokenizer(
        examples["text"],
        truncation=True,
        max_length=BLOCK_SIZE,
        padding=False
    )

In [ ]:
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["text"],
    desc="Токенизация"
)

Токенизация (num_proc=4):   0%|          | 0/2042 [00:00<?, ? examples/s]

In [ ]:
def group_texts(examples):
    """
    Группирует токенизированные тексты в блоки фиксированной длины.
    Это необходимо для эффективного обучения GPT-2.
    """
    # конкатенируем все тексты
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    # обрезаем до кратного BLOCK_SIZE
    total_length = (total_length // BLOCK_SIZE) * BLOCK_SIZE

    # разрезаем на блоки
    result = {
        k: [t[i : i + BLOCK_SIZE] for i in range(0, total_length, BLOCK_SIZE)]
        for k, t in concatenated_examples.items()
    }

    # Labels = input_ids для языкового моделирования
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
    desc="Группировка"
)

Группировка (num_proc=4):   0%|          | 0/2042 [00:00<?, ? examples/s]

In [ ]:
# Разделяем 90% train / 10% test
lm_dataset_split = lm_dataset.train_test_split(test_size=0.1, seed=42)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

print(f"Размер embeddings до: {model.get_input_embeddings().weight.shape[0]}")
model.resize_token_embeddings(len(rap_tokenizer.tokenizer))
print(f"Размер embeddings после: {model.get_input_embeddings().weight.shape[0]}")
model.config.pad_token_id = model.config.eos_token_id

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Размер embeddings до: 50264
Размер embeddings после: 50280


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=rap_tokenizer.tokenizer,
    mlm=False  # Causal LM, не masked LM
)

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,

    # Оптимизация
    learning_rate=LEARNING_RATE,
    warmup_steps=100,
    weight_decay=0.01,

    # Логирование
    logging_dir=LOGS_DIR,
    logging_steps=50,

    # Сохранение
    save_strategy="epoch",
    save_total_limit=3,  # Храним только последние 3 чекпоинта

    # Оценка
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    # Производительность
    fp16=torch.cuda.is_available(),  # Mixed precision для GPU
    gradient_accumulation_steps=2,    # Виртуальный batch_size x2

    # Отчёты
    report_to="none",  # Отключаем W&B/TensorBoard

    # Детерминизм
    seed=42,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset_split["train"],
    eval_dataset=lm_dataset_split["test"],
    data_collator=data_collator,
)

In [50]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.922500,3.117203
2,2.858500,3.116437
3,2.898100,3.114838
4,2.920700,3.113400
5,2.898900,3.108509
6,2.906700,3.107843
7,2.914900,3.105100


Epoch,Training Loss,Validation Loss
1,2.922500,3.117203
2,2.858500,3.116437
3,2.898100,3.114838
4,2.920700,3.113400
5,2.898900,3.108509
6,2.906700,3.107843
7,2.914900,3.105100
8,2.903700,3.103937
9,2.918000,3.102557
10,2.882700,3.102190


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1067, training_loss=2.898829924021241, metrics={'train_runtime': 1427.872, 'train_samples_per_second': 5.978, 'train_steps_per_second': 0.747, 'total_flos': 4460777570304000.0, 'train_loss': 2.898829924021241, 'epoch': 11.0})

In [56]:
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

trainer.save_model(FINAL_MODEL_DIR)
rap_tokenizer.tokenizer.save_pretrained(FINAL_MODEL_DIR)

('./neiroRap_final_model_v2/tokenizer_config.json',
 './neiroRap_final_model_v2/special_tokens_map.json',
 './neiroRap_final_model_v2/vocab.json',
 './neiroRap_final_model_v2/merges.txt',
 './neiroRap_final_model_v2/added_tokens.json',
 './neiroRap_final_model_v2/tokenizer.json')

In [53]:
import shutil
from google.colab import files

folder_to_download = 'neiroRap_final_model_v2'

zip_filename = f'{folder_to_download}.zip'
shutil.make_archive(folder_to_download, 'zip', folder_to_download)

files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>